In [1]:
from ase import Atoms
import ase.units as units
from ase.io import read
from ase.build import bulk
from ase.spacegroup import crystal
from ase.calculators.espresso import Espresso

import numpy as np
import matplotlib.pyplot as plt

import os
import dotenv

dotenv.load_dotenv()
PWSCF_COMMAND = os.environ.get("PWSCF_COMMAND")
PPDIR = os.environ.get("PSEUDOPOTENTIALS")

outdir = "./results"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [3]:
from ase.io import write
from ase import Atoms
from alchemist.qe.utils.objects import Struc, PseudoPotential, Constraint, Kpoints, Dir, ase2struc
from alchemist.qe.utils.pwscf import PWscf_inparam, run_qe_pwscf, parse_qe_pwscf_output


def make_struc(a,c):
    unit_cell = Atoms(symbols=['Au', 'Cu'], positions=[(0, 0, 0), (a/2, a/2, c/2)], 
                    cell=[(a, 0, 0), (0, a, 0), (0, 0, c)], pbc=True)

    # Visualize the unit cell
    # ase.visualize.view(unit_cell, viewer='x3d')

    structure = Struc(ase2struc(unit_cell))
    return structure

def make_fcc(a, element):
    cu_fcc = bulk(element, 'fcc', a=a)
    return Struc(ase2struc(cu_fcc))

def compute_energy(alat, clat, nk, ecut):
    """
    Make an input template and select potential and structure, and the path where to run
    """
    pseudopots = {'Au': PseudoPotential(ptype='uspp', element='Au', functional='LDA', name='Au.pz-d-rrkjus.UPF', path=PPDIR+'Au.pz-d-rrkjus.UPF'),
                  'Cu': PseudoPotential(ptype='uspp', element='Cu', functional='LDA', name='Cu.pz-d-rrkjus.UPF', path=PPDIR+'Cu.pz-d-rrkjus.UPF')}
    struc = make_struc(alat, clat)
    # fix the Pb and Ti atoms in place during relaxation
    # constraint = Constraint(atoms={'0': [0,0,0], '1': [0,0,0]})
    kpts = Kpoints(gridsize=[nk, nk, nk], option='automatic', offset=False)
    runpath = Dir(path="./results")
    input_params = PWscf_inparam({
        'CONTROL': {
            'calculation': 'vc-relax',
            'pseudo_dir': PPDIR,
            'outdir': runpath.path,
            'tstress': True,
            'tprnfor': True,
            'disk_io': 'none'
        },
        'SYSTEM': {
            'ecutwfc': ecut,
            'ecutrho': ecut * 8,
            'occupations': 'smearing',
            'smearing': 'mp',
            'degauss': 0.02,
             },
        'ELECTRONS': {
            'diagonalization': 'david',
            'mixing_beta': 0.2,
            'conv_thr': 1e-7,
        },
        'IONS': {
            'ion_dynamics': 'bfgs'
        },
        'CELL': {'cell_dynamics': 'bfgs'},
        })

    output_file = run_qe_pwscf(runpath=runpath, struc=struc,  pseudopots=pseudopots,
                               params=input_params, kpoints=kpts, ncpu=16)
    output = parse_qe_pwscf_output(outfile=output_file)
    return output



# out = compute_energy(3.97, 3, 30, 0.1)

ModuleNotFoundError: No module named 'alchemist.qe'

In [2]:
energies = []
for k in range(3, 8):
    out = compute_energy(3, 3.3, k, 40)
    print (out)
    energies.append(out['energy'])

In [ ]:

# Given energies in meV
kpts = [3, 4, 5, 6, 7]

# Conversion factor: 1 meV = 0.0367493 millihartree
energies_millihartree = [energy * 0.0367493 for energy in energies]

# Generating the plot
plt.plot([i for i in range(3, 8)], energies_millihartree, marker='o', linestyle='-', color='blue')

# Adding plot labels and title for clarity
plt.xlabel('k-points')
plt.ylabel('Energy (mHa)')
plt.title('Energy vs. k-points')
plt.grid(True)

# Show the improved plot
plt.show()


In [ ]:
final_structure = read('results/pwscf.out') 

# Extracting the lattice constants
lattice_constants = final_structure.cell.lengths()
print (lattice_constants)